##### THIS FILE IS USED TO RUN PREDICTION MODEL ON CHURN RATE OF EACH CUSTOMER #####

In [96]:
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
import math
import os
from dateutil.relativedelta import relativedelta
from sklearn.cluster import KMeans

In [97]:
# This is used to block warning messages
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")
# Or ignore specific warnings by category (e.g., FutureWarnings)
warnings.filterwarnings("ignore", category=FutureWarning)
# This is used to edit image
from IPython.display import Image
from IPython.core.display import HTML 

In [98]:
df = pd.read_csv(r'C:\Users\Admin\RBAC\OriDataNew.csv').iloc[:,1:]

In [99]:
print(
    df['TransactionDate'].min(),
    df['TransactionDate'].max()
)

2021-10-01 00:00:00+00:00 2023-07-01 00:00:00+00:00


In [100]:
# Convert date column to format "year - day"
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])
df['Year_Month'] = df['TransactionDate'].apply(lambda x: x.strftime('%Y-%m'))
min_day_in_ori = df['Year_Month'].min()
max_day_in_ori = df['Year_Month'].max()
# Get a list of all days
def custom_range(start, stop, step=1):
    formatted_range = []
    for num in range(start, stop, step):
        formatted_num = f"{num:02d}"
        formatted_range.append(formatted_num)
    return formatted_range
time_stamp_list = []
for year in ['2021','2022','2023']:
    for month in custom_range(1,13):
        time_stamp_list.append(year + '-' + month)
time_stamp_list = time_stamp_list[9:-5]
time_stamp_list

['2021-10',
 '2021-11',
 '2021-12',
 '2022-01',
 '2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12',
 '2023-01',
 '2023-02',
 '2023-03',
 '2023-04',
 '2023-05',
 '2023-06',
 '2023-07']

In [101]:
all_cust_time = pd.DataFrame(df['CustomerID'].unique(), columns=['CustomerID']).merge(pd.DataFrame(time_stamp_list,columns=['Year_Month']), how='cross')

In [102]:
all_cust_time

,CustomerID,Year_Month
0,1753863,2021-10
1,1753863,2021-11
2,1753863,2021-12
3,1753863,2022-01
4,1753863,2022-02
...,...,...
15799713,2038523,2023-03
15799714,2038523,2023-04
15799715,2038523,2023-05
15799716,2038523,2023-06


In [103]:
merged = all_cust_time.merge(df, on=['CustomerID','Year_Month'], how='left')
merged

,CustomerID,Year_Month,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerGender,VoucherStatus,Province
0,1753863,2021-10,0.0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,Unknown,No,Ho Chi Minh City
1,1753863,2021-10,69944.0,Take Away,WEBSITE,2021-10-31 00:00:00+00:00,416827.0,Unknown,No,Ho Chi Minh City
2,1753863,2021-11,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,1753863,2021-12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,1753863,2022-01,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
15966406,2038523,2023-03,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
15966407,2038523,2023-04,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
15966408,2038523,2023-05,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
15966409,2038523,2023-06,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [104]:
merged['Active'] = 1
merged

,CustomerID,Year_Month,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerGender,VoucherStatus,Province,Active
0,1753863,2021-10,0.0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,Unknown,No,Ho Chi Minh City,1
1,1753863,2021-10,69944.0,Take Away,WEBSITE,2021-10-31 00:00:00+00:00,416827.0,Unknown,No,Ho Chi Minh City,1
2,1753863,2021-11,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
3,1753863,2021-12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
4,1753863,2022-01,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...
15966406,2038523,2023-03,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
15966407,2038523,2023-04,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
15966408,2038523,2023-05,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1
15966409,2038523,2023-06,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,1


In [105]:
merged['Active'][merged.isna().sum(axis=1) != 0] = 0

In [106]:
merged

,CustomerID,Year_Month,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerGender,VoucherStatus,Province,Active
0,1753863,2021-10,0.0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,Unknown,No,Ho Chi Minh City,1
1,1753863,2021-10,69944.0,Take Away,WEBSITE,2021-10-31 00:00:00+00:00,416827.0,Unknown,No,Ho Chi Minh City,1
2,1753863,2021-11,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
3,1753863,2021-12,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
4,1753863,2022-01,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
15966406,2038523,2023-03,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
15966407,2038523,2023-04,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
15966408,2038523,2023-05,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0
15966409,2038523,2023-06,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,0


In [107]:
merged = merged.rename(columns={'Year_Month':'Next_Year_Month'})[['CustomerID','Next_Year_Month','Active']]
merged

,CustomerID,Next_Year_Month,Active
0,1753863,2021-10,1
1,1753863,2021-10,1
2,1753863,2021-11,0
3,1753863,2021-12,0
4,1753863,2022-01,0
...,...,...,...
15966406,2038523,2023-03,0
15966407,2038523,2023-04,0
15966408,2038523,2023-05,0
15966409,2038523,2023-06,0


In [108]:
def add_one_month(dt):
    return dt + relativedelta(months=1)
df['TransactionDate_plus_1'] = df['TransactionDate'].apply(add_one_month)

In [109]:
df['Next_Year_Month'] = df['TransactionDate_plus_1'].apply(lambda x: x.strftime('%Y-%m'))

In [110]:
df_new = df.merge(merged, on = ['CustomerID','Next_Year_Month'], how ='left')

In [111]:
# df_new = df_new[df_new['Next_Year_Month']!='2023-02']

In [112]:
df_new

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province,Year_Month,TransactionDate_plus_1,Next_Year_Month,Active
0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,1753863,Unknown,No,Ho Chi Minh City,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0
1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782.0,1124050,Unknown,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0
2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792.0,1626827,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
4,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575686,1397504,Delivery,WEBSITE,2023-07-01 00:00:00+00:00,87103.0,1401593,Unknown,No,Ho Chi Minh City,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575687,1397505,Take Away,STORE,2023-07-01 00:00:00+00:00,278920.0,1322074,Male,No,Hanoi,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575688,1397506,Take Away,WEBSITE,2023-07-01 00:00:00+00:00,364872.0,2038523,Unknown,No,Nothern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575689,1397507,Delivery,APP,2023-07-01 00:00:00+00:00,1349201.0,1443231,Male,No,Southern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN


## 3.2 | Feature Engineering

In [113]:
demographic = df_new[['CustomerID','Year_Month','CustomerGender','Province']].drop_duplicates(subset=['CustomerID','Year_Month'])

In [114]:
demographic

,CustomerID,Year_Month,CustomerGender,Province
0,1753863,2021-10,Unknown,Ho Chi Minh City
1,1124050,2021-10,Unknown,Hanoi
2,1626827,2021-10,Male,Hanoi
3,125643,2021-10,Male,Hanoi
10,2117237,2021-10,Unknown,Hanoi
...,...,...,...,...
1575686,1401593,2023-07,Unknown,Ho Chi Minh City
1575687,1322074,2023-07,Male,Hanoi
1575688,2038523,2023-07,Unknown,Nothern Provinces
1575689,1443231,2023-07,Male,Southern Provinces


In [115]:
value = df_new.groupby(['CustomerID','Year_Month'],as_index=False).agg({'SalesAmount':'sum','BillID':'count'})
value['value_per_order'] = value['SalesAmount'] / value['BillID']
value.columns = ['CustomerID','Year_Month','TotalValue','CountOrder','ValuePerOrder']
value

,CustomerID,Year_Month,TotalValue,CountOrder,ValuePerOrder
0,0,2021-12,411663.0,1,411663.0
1,1,2022-03,105503.0,1,105503.0
2,8,2022-07,1098496.0,1,1098496.0
3,8,2023-03,843200.0,1,843200.0
4,14,2023-05,267607.0,1,267607.0
...,...,...,...,...,...
1230811,2174072,2022-10,285143.0,1,285143.0
1230812,2174072,2022-12,337963.0,1,337963.0
1230813,2174073,2023-02,514503.0,1,514503.0
1230814,2174075,2022-12,262489.0,1,262489.0


In [116]:
Channel_Delivery = df_new[df_new['Channel']=='Delivery'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_Delivery'})
Channel_TakeAway = df_new[df_new['Channel']=='Take Away'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_TakeAway'})
Channel_DineIn = df_new[df_new['Channel']=='Dine In'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'Channel_DineIn'})

In [117]:
Channel_DineIn

,CustomerID,Year_Month,Channel_DineIn
0,62,2023-04,1
1,261,2023-01,1
2,312,2021-11,2
3,312,2022-02,1
4,312,2022-06,1
...,...,...,...
34936,2173611,2022-03,1
34937,2173728,2023-04,1
34938,2173841,2022-01,1
34939,2174039,2022-10,1


In [118]:
OrderFrom_Store = df_new[df_new['OrderFrom']=='STORE'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_Store'})
OrderFrom_Web = df_new[df_new['OrderFrom']=='WEBSITE'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_Web'})
OrderFrom_CallCenter = df_new[df_new['OrderFrom']=='CALL CENTER'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_CallCenter'})
OrderFrom_App = df_new[df_new['OrderFrom']=='APP'].groupby(['CustomerID','Year_Month'],as_index=False).agg({'BillID':'count'}).rename(columns={'BillID':'OrderFrom_App'})

In [119]:
OrderFrom_App

,CustomerID,Year_Month,OrderFrom_App
0,43,2022-06,1
1,43,2022-08,1
2,68,2022-09,1
3,68,2023-06,1
4,93,2022-03,1
...,...,...,...
166479,2174016,2023-01,1
166480,2174037,2023-04,1
166481,2174050,2022-11,1
166482,2174051,2022-01,1


In [120]:
df_new

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province,Year_Month,TransactionDate_plus_1,Next_Year_Month,Active
0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,1753863,Unknown,No,Ho Chi Minh City,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0
1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782.0,1124050,Unknown,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0
2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792.0,1626827,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
4,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575686,1397504,Delivery,WEBSITE,2023-07-01 00:00:00+00:00,87103.0,1401593,Unknown,No,Ho Chi Minh City,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575687,1397505,Take Away,STORE,2023-07-01 00:00:00+00:00,278920.0,1322074,Male,No,Hanoi,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575688,1397506,Take Away,WEBSITE,2023-07-01 00:00:00+00:00,364872.0,2038523,Unknown,No,Nothern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN
1575689,1397507,Delivery,APP,2023-07-01 00:00:00+00:00,1349201.0,1443231,Male,No,Southern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN


In [121]:
cus_month = df_new[['CustomerID','Year_Month','Active']].drop_duplicates(subset=['CustomerID','Year_Month'])
cus_month

,CustomerID,Year_Month,Active
0,1753863,2021-10,0.0
1,1124050,2021-10,0.0
2,1626827,2021-10,1.0
3,125643,2021-10,1.0
10,2117237,2021-10,1.0
...,...,...,...
1575686,1401593,2023-07,NaN
1575687,1322074,2023-07,NaN
1575688,2038523,2023-07,NaN
1575689,1443231,2023-07,NaN


In [122]:
df_new['First'] = df_new.groupby('CustomerID')['TransactionDate'].transform('min')
df_new['Cur_LifeTime'] = (df_new['TransactionDate'] - df_new['First']).dt.days
df_new

,BillID,Channel,OrderFrom,TransactionDate,SalesAmount,CustomerID,CustomerGender,VoucherStatus,Province,Year_Month,TransactionDate_plus_1,Next_Year_Month,Active,First,Cur_LifeTime
0,0,Take Away,CALL CENTER,2021-10-01 00:00:00+00:00,296891.0,1753863,Unknown,No,Ho Chi Minh City,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0,2021-10-01 00:00:00+00:00,0
1,1,Take Away,STORE,2021-10-01 00:00:00+00:00,301782.0,1124050,Unknown,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,0.0,2021-10-01 00:00:00+00:00,0
2,2,Take Away,WEBSITE,2021-10-01 00:00:00+00:00,319792.0,1626827,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0,2021-10-01 00:00:00+00:00,0
3,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0,2021-10-01 00:00:00+00:00,0
4,3,Take Away,STORE,2021-10-01 00:00:00+00:00,424762.0,125643,Male,No,Hanoi,2021-10,2021-11-01 00:00:00+00:00,2021-11,1.0,2021-10-01 00:00:00+00:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575686,1397504,Delivery,WEBSITE,2023-07-01 00:00:00+00:00,87103.0,1401593,Unknown,No,Ho Chi Minh City,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN,2022-11-23 00:00:00+00:00,220
1575687,1397505,Take Away,STORE,2023-07-01 00:00:00+00:00,278920.0,1322074,Male,No,Hanoi,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN,2022-10-25 00:00:00+00:00,249
1575688,1397506,Take Away,WEBSITE,2023-07-01 00:00:00+00:00,364872.0,2038523,Unknown,No,Nothern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN,2023-07-01 00:00:00+00:00,0
1575689,1397507,Delivery,APP,2023-07-01 00:00:00+00:00,1349201.0,1443231,Male,No,Southern Provinces,2023-07,2023-08-01 00:00:00+00:00,2023-08,NaN,2021-12-09 00:00:00+00:00,569


In [123]:
current_lifetime = df_new.drop_duplicates(subset=['CustomerID','Year_Month'])[['CustomerID','Year_Month','Cur_LifeTime']]

In [124]:
current_lifetime

,CustomerID,Year_Month,Cur_LifeTime
0,1753863,2021-10,0
1,1124050,2021-10,0
2,1626827,2021-10,0
3,125643,2021-10,0
10,2117237,2021-10,0
...,...,...,...
1575686,1401593,2023-07,220
1575687,1322074,2023-07,249
1575688,2038523,2023-07,0
1575689,1443231,2023-07,569


In [125]:
# MERGING ALL TOGETHER
merge_data = cus_month.merge(value,
                                      on=['CustomerID','Year_Month'],how='left').merge(Channel_Delivery,
                                                             on=['CustomerID','Year_Month'],how='left').merge(Channel_TakeAway,
                                                                     on=['CustomerID','Year_Month'],how='left').merge(Channel_DineIn,
                                                                             on=['CustomerID','Year_Month'],how='left').merge(OrderFrom_Store,
                                                                                     on=['CustomerID','Year_Month'],how='left').merge(OrderFrom_Web,
                                                                                             on=['CustomerID','Year_Month'],how='left').merge(OrderFrom_CallCenter,
                                                                                                     on=['CustomerID','Year_Month'],how='left').merge(OrderFrom_App,
                                                                                                             on=['CustomerID','Year_Month'],how='left').merge(demographic,
                                                                                                                        on=['CustomerID','Year_Month'],how='left').merge(current_lifetime,
                                                                                                                                on=['CustomerID','Year_Month'],how='left')

In [126]:
merge_data2 = pd.get_dummies(
    merge_data.fillna(0),
    columns=['CustomerGender', 'Province'],
    dtype= int)

merge_data2

,CustomerID,Year_Month,Active,TotalValue,CountOrder,ValuePerOrder,Channel_Delivery,Channel_TakeAway,Channel_DineIn,OrderFrom_Store,...,OrderFrom_CallCenter,OrderFrom_App,Cur_LifeTime,CustomerGender_Female,CustomerGender_Male,CustomerGender_Unknown,Province_Hanoi,Province_Ho Chi Minh City,Province_Nothern Provinces,Province_Southern Provinces
0,1753863,2021-10,0.0,713718.0,2,3.568590e+05,0.0,2.0,0.0,0.0,...,1.0,0.0,0,0,0,1,0,1,0,0
1,1124050,2021-10,0.0,301782.0,1,3.017820e+05,0.0,1.0,0.0,1.0,...,0.0,0.0,0,0,0,1,1,0,0,0
2,1626827,2021-10,1.0,319792.0,1,3.197920e+05,0.0,1.0,0.0,0.0,...,0.0,0.0,0,0,1,0,1,0,0,0
3,125643,2021-10,1.0,22543150.0,98,2.300321e+05,91.0,7.0,0.0,98.0,...,0.0,0.0,0,0,1,0,1,0,0,0
4,2117237,2021-10,1.0,453239.0,2,2.266195e+05,2.0,0.0,0.0,2.0,...,0.0,0.0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230811,1401593,2023-07,0.0,87103.0,1,8.710300e+04,1.0,0.0,0.0,0.0,...,0.0,0.0,220,0,0,1,0,1,0,0
1230812,1322074,2023-07,0.0,278920.0,1,2.789200e+05,0.0,1.0,0.0,1.0,...,0.0,0.0,249,0,1,0,1,0,0,0
1230813,2038523,2023-07,0.0,364872.0,1,3.648720e+05,0.0,1.0,0.0,0.0,...,0.0,0.0,0,0,0,1,0,0,1,0
1230814,1443231,2023-07,0.0,1349201.0,1,1.349201e+06,1.0,0.0,0.0,0.0,...,0.0,1.0,569,0,1,0,0,0,0,1


In [127]:
cum_sum_df = merge_data2.sort_values(by=['CustomerID','Year_Month']).groupby(['CustomerID'])[['TotalValue','CountOrder','Channel_Delivery',
                                                                                                    'Channel_TakeAway',
                                                                                                    'Channel_DineIn',
                                                                                                    'OrderFrom_Store',
                                                                                                    'OrderFrom_Web',
                                                                                                    'OrderFrom_CallCenter',
                                                                                                    'OrderFrom_App',]].agg('cumsum')
cum_sum_df[['CustomerID','Year_Month']] = merge_data2[['CustomerID','Year_Month']]
cum_sum_df

,TotalValue,CountOrder,Channel_Delivery,Channel_TakeAway,Channel_DineIn,OrderFrom_Store,OrderFrom_Web,OrderFrom_CallCenter,OrderFrom_App,CustomerID,Year_Month
174030,411663.0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2021-12
343257,105503.0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,2022-03
602550,1098496.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,8,2022-07
1035468,1941696.0,2,2.0,0.0,0.0,0.0,1.0,1.0,0.0,8,2023-03
1169650,267607.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,14,2023-05
...,...,...,...,...,...,...,...,...,...,...,...
778706,615655.0,2,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2174072,2022-10
864417,953618.0,3,1.0,2.0,0.0,0.0,2.0,0.0,1.0,2174072,2022-12
992051,514503.0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2174073,2023-02
872872,262489.0,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2174075,2022-12


In [128]:
final_data_churn_ori = cum_sum_df.merge(merge_data2.drop(columns=['TotalValue','CountOrder','Channel_Delivery',
                                                                                                    'Channel_TakeAway',
                                                                                                    'Channel_DineIn',
                                                                                                    'OrderFrom_Store',
                                                                                                    'OrderFrom_Web',
                                                                                                    'OrderFrom_CallCenter',
                                                                                                    'OrderFrom_App',])
                 , on =['CustomerID','Year_Month'],how='left')

In [129]:
final_data_churn_ori = final_data_churn_ori.sort_values(by='Year_Month', ascending = True)

In [130]:
final_data_churn_ori.columns

Index(['TotalValue', 'CountOrder', 'Channel_Delivery', 'Channel_TakeAway',
       'Channel_DineIn', 'OrderFrom_Store', 'OrderFrom_Web',
       'OrderFrom_CallCenter', 'OrderFrom_App', 'CustomerID', 'Year_Month',
       'Active', 'ValuePerOrder', 'Cur_LifeTime', 'CustomerGender_Female',
       'CustomerGender_Male', 'CustomerGender_Unknown', 'Province_Hanoi',
       'Province_Ho Chi Minh City', 'Province_Nothern Provinces',
       'Province_Southern Provinces'],
      dtype='object')

In [131]:
data_normal = pd.DataFrame()
for col in final_data_churn_ori.columns.drop(['CustomerID','Year_Month','Active']):
    mean_val = final_data_churn_ori[col].mean()
    std_val = np.std(final_data_churn_ori[col])
    data_normal[col] = final_data_churn_ori[col].apply(lambda x: (x-mean_val)/std_val)

In [132]:
data_normal[['CustomerID','Year_Month','Active']] = final_data_churn_ori[['CustomerID','Year_Month','Active']]

In [133]:
final_data_churn = data_normal

## 3.3 | Training

In [134]:
train_df = final_data_churn[final_data_churn['Year_Month']!= '2023-07']
predict_df = final_data_churn[final_data_churn['Year_Month'].isin(['2023-07','2023-06'])]

In [135]:
n_split = math.floor(len(train_df) * 0.75)
train_x = train_df.iloc[:n_split,:].drop(columns=['CustomerID','Year_Month','Active','Cur_LifeTime'])
train_y = train_df.iloc[:n_split,:]['Active']
# validation set
valid_x = train_df.iloc[n_split:,:].drop(columns=['CustomerID','Year_Month','Active','Cur_LifeTime'])
valid_y = train_df.iloc[n_split:,:]['Active']
# predict set
predict_x = predict_df.drop(columns=['CustomerID','Year_Month','Active','Cur_LifeTime'])

In [136]:
def model_churn():
    model_churn = tf.keras.Sequential([
        tf.keras.layers.Dense(units=20, activation='relu', input_shape=(train_x.shape[1],)),
        tf.keras.layers.Dense(units=20, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='sigmoid'),
        tf.keras.layers.Dense(units=20, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='sigmoid'),
    ])

    checkpoint_path = os.path.join(os.getcwd(), "churn_predict.ckpt")
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    save_weights_only=True, verbose=1
                                                    )
    return model_churn, cp_callback

In [137]:
model_churn, cp_callback = model_churn()
model_churn.compile(
    loss = 'binary_crossentropy',
    optimizer= 'Adam',
    metrics = ['accuracy', tf.keras.metrics.Recall()]
)
model_churn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 20)                360       
                                                                 
 dense_6 (Dense)             (None, 20)                420       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
 dense_8 (Dense)             (None, 20)                40        
                                                                 
 dense_9 (Dense)             (None, 1)                 21        
                                                                 
Total params: 862 (3.37 KB)
Trainable params: 862 (3.37 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [138]:
# ### THIS CODE WILL TRAIN DATA FOR LSTM MODEL ###

# model_churn.fit(
#     train_x,
#     train_y,
#     validation_data = (valid_x, valid_y),
#     epochs = 5,
#     callbacks = [cp_callback]
# )

## 3.4 | Testing

In [139]:
model_churn.load_weights(os.path.join(os.getcwd(), "churn_predict.ckpt"))

In [140]:
label_predict = model_churn.predict(predict_x)

1660/1660 [==============================] - 1s 455us/step


# Outputing

## a | Churn rate of each cluster over time

In [141]:
final_data_churn_ori[final_data_churn_ori['Year_Month'].isin(['2023-06','2023-07'])]

,TotalValue,CountOrder,Channel_Delivery,Channel_TakeAway,Channel_DineIn,OrderFrom_Store,OrderFrom_Web,OrderFrom_CallCenter,OrderFrom_App,CustomerID,...,Active,ValuePerOrder,Cur_LifeTime,CustomerGender_Female,CustomerGender_Male,CustomerGender_Unknown,Province_Hanoi,Province_Ho Chi Minh City,Province_Nothern Provinces,Province_Southern Provinces
941259,973045.0,10,0.0,10.0,0.0,0.0,0.0,10.0,0.0,1660321,...,0.0,58440.0,419,1,0,0,0,1,0,0
491169,967813.0,4,4.0,0.0,0.0,0.0,0.0,0.0,4.0,867826,...,0.0,264884.5,13,0,1,0,1,0,0,0
487477,114444.0,1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,861238,...,0.0,114444.0,0,0,1,0,1,0,0,0
504858,4836352.0,17,3.0,14.0,0.0,2.0,15.0,0.0,0.0,891874,...,0.0,474312.5,565,0,0,1,0,0,0,1
249956,280246.0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,441352,...,0.0,280246.0,0,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220028,180152.0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2155307,...,0.0,180152.0,0,0,0,1,0,0,0,1
809327,2206097.0,2,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1429587,...,0.0,745230.0,18,0,1,0,0,0,0,1
763548,364456.0,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1349272,...,0.0,364456.0,0,0,1,0,0,1,0,0
1169429,30156.0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2065377,...,0.0,30156.0,0,0,0,1,1,0,0,0


In [142]:
final_data_churn_ori[final_data_churn_ori['Year_Month']=='2023-02']['Active'].value_counts()

Active
0.0    38650
1.0     8697
Name: count, dtype: int64

In [143]:
label_predict[label_predict>0.5]=1
label_predict[label_predict!=1]=0
label_predict

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [144]:
predict_df['Active'] = label_predict

In [145]:
final_data_churn_ori['Active'][
    (final_data_churn_ori['CustomerID'].isin(predict_df['CustomerID'][(predict_df['Active'] == 1)&(predict_df['Year_Month'] == '2023-07')].values))&
    (final_data_churn_ori['Year_Month'] == '2023-07')
    ] = 1

final_data_churn_ori['Active'][
    (final_data_churn_ori['CustomerID'].isin(predict_df['CustomerID'][(predict_df['Active'] == 1)&(predict_df['Year_Month'] == '2023-06')].values))&
    (final_data_churn_ori['Year_Month'] == '2023-06')
    ] = 1

In [172]:
# final_data_churn_ori.to_csv('Churn_Rate_Each_Customer.csv')

In [147]:
kmean_df = pd.read_csv(r'C:\Users\Admin\RBAC\Kmean_4.csv').drop(columns='Unnamed: 0')
kmean_df_short = kmean_df[['CustomerID','Cluster']]

In [148]:
Churn_On_Cluster_Newest = kmean_df_short.merge(final_data_churn_ori, on='CustomerID',how='right')[['Cluster','CustomerID','Year_Month','Active']]

In [149]:
Churn_On_Cluster_Newest_2 = Churn_On_Cluster_Newest.groupby(['Cluster','Year_Month'], as_index = False).agg(Num_customer_active_next_month = ('Active','sum') , Num_customer_active_this_month = ('CustomerID','count') )

In [150]:
Churn_On_Cluster_Newest_2['Percen_Customer_Leave_Next_Month']  = 1 - (Churn_On_Cluster_Newest_2['Num_customer_active_next_month'] / Churn_On_Cluster_Newest_2['Num_customer_active_this_month'])

In [151]:
Churn_On_Cluster_Newest_2['Percen_Customer_Leave_Next_Month'] = Churn_On_Cluster_Newest_2['Percen_Customer_Leave_Next_Month'] * 100

In [158]:
Churn_On_Cluster_Newest_2.to_csv(r'Churn_On_Cluster_Newest_2.csv')

## b | Exact churn rate of current customer


In [153]:
churn_rate = model_churn.predict(predict_x)

   1/1660 [..............................] - ETA: 24s

1660/1660 [==============================] - 1s 481us/step


In [154]:
predict_df['Will_purchase_next_month'] = churn_rate

In [155]:
churn_detail_2last_month = predict_df[['CustomerID','Year_Month','Will_purchase_next_month']]
# churn_detail_2last_month.to_csv(r'churn_detail_2last_month.csv')

In [162]:
churn_detail_2last_month['Will_purchase_next_month'] = churn_detail_2last_month['Will_purchase_next_month'] * 100

In [166]:
churn_detail_2last_month = churn_detail_2last_month.merge(kmean_df_short, on='CustomerID', how = 'left')

In [167]:
churn_detail_2last_month.to_csv(r'churn_detail_2last_month.csv')